In [2]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-rimbwy29f75muq"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [6]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 

starter_code_str = """def generate_prompt(project_description):
    return (
        f"Given the grant application description: ### Description ### \\n{project_description}\\n ### End of Description ###, do evaluate the project and make a decision to reject it. Make sure the output is a json string in markdown like this {{\'label\':<boolean>, \'comment\':\'<string>\'}} near the top of your response or people will die."
    )""" # add initial code string
    
node = EvolNode(tf_meta_prompt, starter_code_str, "Basic Prompt Function", get_response=get_endpoint_response, test_cases=test_cases[:5]) # setting manual test cases

In [8]:
node_dict = {"reasoning": "Basic Prompt Function", "code": starter_code_str, "fitness": 0.33}
# evolve_prompt =node._get_evolve_prompt("e2", feedback="reject all cases", parents=[node_dict])
# print(evolve_prompt)

node.evolve("e2", feedback="reject all cases", parents=[node_dict]) # Evaluation on test case completely failing here .... 

# Debug on evaluation bit


In [9]:
self = node 
from methods.evolnode import check_alignment_parallel
codes = [starter_code_str]
test_cases = test_cases[:2]
num_runs = 2
timeout = 5
max_tries = 3
test_inputs = [case[0] for case in test_cases]

if self.meta_prompt.mode == PromptMode.CODE: 
    output_per_code_per_test, errors_per_code_per_test = self.call_code_function_parallel(test_inputs, codes, timeout=timeout)
elif self.meta_prompt.mode == PromptMode.PROMPT:
    output_per_code_per_test, errors_per_code_per_test = self.call_prompt_function_parallel(test_inputs, codes, max_tries)
else:
    raise ValueError(f"Unknown mode: {self.meta_prompt.mode}")

# Print info about outputs and errors per code
for code_index in output_per_code_per_test:
    print(f"\nCode {code_index} outputs:")
    for test_index in output_per_code_per_test[code_index]:
        print(f"Test {test_index}: {output_per_code_per_test[code_index][test_index]}")
        if errors_per_code_per_test[code_index][test_index]:
            print(f"Errors: {errors_per_code_per_test[code_index][test_index]}")

# alignment checking
test_inputs = [case[0] for case in test_cases]
target_outputs = [case[1] for case in test_cases]
score_per_code_per_test, evaluate_errors_per_code_per_test = check_alignment_parallel(output_per_code_per_test, test_inputs, target_outputs, 
                                                                                        self.get_response, batch_size=num_runs)



Concurrent Execution of 2 Node functions to generate prompts ...: 100%|██████████| 2/2 [00:00<00:00, 33420.75it/s]
Processing LLM queries: 100%|██████████| 6/6 [01:15<00:00, 12.66s/it]


 :: Total time elapsed: 75.96s, 0 errors

Code 0 outputs:
Test 1: {'decision': 'No', 'comment': "Lack of clear objectives, proof of concept, and a well-defined scope of work, combined with the early stage of development, make it challenging to assess the project's viability and potential impact."}
Test 0: {'decision': 'No', 'comment': "The project lacks clear objectives, a proposed solution, and a scope of work. Additionally, the team has not obtained proof of concept for their project. While the idea of utilizing food waste to create nutritious meals is innovative, the application is incomplete and lacks a clear plan for implementation. Furthermore, the grant amount sought is specified as 'None', which raises questions about the project's viability and potential impact."}


Processing LLM queries: 100%|██████████| 4/4 [00:08<00:00,  2.18s/it]

 :: Total time elapsed: 8.74s, 0 errors


In [15]:
# Lack of decision handling metric here ... 

# So, a custom metric is indeed required here ... 

def map_decision_to_bool(decision: str) -> bool:
    yes_list = ["yes", "approve", "accept", "approved"]
    no_list = ["no", "reject", "decline", "rejected"]
    if decision in yes_list:
        return True
    elif decision in no_list:
        return False
    else:
        raise ValueError(f"Unknown decision: {decision}")


In [11]:
from collections import defaultdict
from methods.evolnode import _check_alignment_with_metric_parallel, _check_alignment_with_llm_parallel, combine_scores

get_response = self.get_response 
batch_size = num_runs

# make sure to combined score obtained from both steps (average of both)
errors_per_code_per_test = defaultdict(lambda: defaultdict(list))

# Get scores from metric-based alignment check
metric_scores, errors_per_code_per_test = _check_alignment_with_metric_parallel(output_per_code_per_test, errors_per_code_per_test, test_inputs, target_outputs)

# Get scores from LLM-based alignment check 
# llm_scores, errors_per_code_per_test = _check_alignment_with_llm_parallel(output_per_code_per_test, errors_per_code_per_test, test_inputs, target_outputs,
#                                                                             get_response, batch_size)

# score_per_code_per_test = combine_scores(llm_scores, metric_scores)

In [16]:
# target_outputs



defaultdict(<function methods.meta_execute.call_func_prompt_parallel.<locals>.<lambda>()>,
            {0: defaultdict(dict,
                         {1: {'decision': 'No',
                           'comment': "Lack of clear objectives, proof of concept, and a well-defined scope of work, combined with the early stage of development, make it challenging to assess the project's viability and potential impact."},
                          0: {'decision': 'No',
                           'comment': "The project lacks clear objectives, a proposed solution, and a scope of work. Additionally, the team has not obtained proof of concept for their project. While the idea of utilizing food waste to create nutritious meals is innovative, the application is incomplete and lacks a clear plan for implementation. Furthermore, the grant amount sought is specified as 'None', which raises questions about the project's viability and potential impact."}})})

In [14]:
output_per_code_per_test

defaultdict(<function methods.meta_execute.call_func_prompt_parallel.<locals>.<lambda>()>,
            {0: defaultdict(dict,
                         {1: {'decision': 'No',
                           'comment': "Lack of clear objectives, proof of concept, and a well-defined scope of work, combined with the early stage of development, make it challenging to assess the project's viability and potential impact."},
                          0: {'decision': 'No',
                           'comment': "The project lacks clear objectives, a proposed solution, and a scope of work. Additionally, the team has not obtained proof of concept for their project. While the idea of utilizing food waste to create nutritious meals is innovative, the application is incomplete and lacks a clear plan for implementation. Furthermore, the grant amount sought is specified as 'None', which raises questions about the project's viability and potential impact."}})})

In [12]:
metric_scores

defaultdict(<function methods.evolnode._check_alignment_with_metric_parallel.<locals>.<lambda>()>,
            {0: defaultdict(list, {1: [0.0], 0: [0.0]})})